# INF-510 Computational Numerical Methods
## Fire spread model based on Linear Reaction-Convection-Diffusion Equation
### Daniel San Martín

## Linear Reaction-Convection-Diffusion Equation

The linear reaction-convection-diffusion [1] equation is defined by

\begin{equation}
    \frac{\partial u}{\partial t} = \mu \nabla^2u - \nabla \cdot (u\textbf{v})+au,
\end{equation}

where $u$ can represent the temperature or concentration of chemical species in a domain $\Omega \in \mathbb{R}^n$, $\mu>0$ is the diffusion constant, $\textbf{v}(\textbf{x})=(v_1(\textbf{x}), ..., v_n(\textbf{x}))$ is a velocity field of flow fluid and $a(\textbf{x})$ is a reaction rate.

### Diffusion term

The model is based in the process of particles movement and this term is defined by 

\begin{equation}
    \begin{split}
        \mu \nabla^2u & = \mu\left(\frac{\partial^2 u}{\partial x_1^2} 
            + ... + \frac{\partial^2 u}{\partial x_n^2}\right). \\
    \end{split}
\end{equation}

For $\textbf{x}\in\mathbb{R}^2$ we have
\begin{equation}
    \mu \nabla^2u = \mu\left(\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2}\right).
\end{equation}

### Convection term

The model includes the effect of a vector field $\textbf{v}(\textbf{x})=(v_1(\textbf{x}), ..., v_n(\textbf{x}))$ in the form 

\begin{equation}
    \begin{split}
        \nabla \cdot (u\textbf{v}) & = \sum_{i=1}^n\frac{\partial(uv_i)}{\partial x_i}.
    \end{split}
\end{equation}

For $\textbf{x}\in\mathbb{R}^2$ we have
\begin{equation}
    \begin{split}
        \nabla \cdot (u\textbf{v}) & = \frac{\partial(uv_1)}{\partial x} + \frac{\partial(uv_2)}{\partial y} \\
         & = \frac{\partial u}{\partial x}v_1 + u\frac{\partial v_1}{\partial x} +
             \frac{\partial u}{\partial y}v_2 + u\frac{\partial v_2}{\partial y}
    \end{split}
\end{equation}

### Reaction term

The reaction term is a linear approximation of chemical kinetics and is defined by $au$, where $a$ may be a real number or a scalar field.

<!--\begin{equation}
    au = a(\textbf{x})u(\textbf{x}).
\end{equation}

For $\textbf{x}\in\mathbb{R}^2$ we have $a(x,y)u(x,y)$-->

## Fire spreading

Assuming that we have a scalar field as fuel and a constant wind, we want to model a fire propagation (heat transport) using the PDE as follow

\begin{equation}
    \begin{split}
        \frac{\partial u}{\partial t} & = \mu \nabla^2u - \nabla \cdot (u\textbf{v})+au ~ \text{in} ~ \Omega \\
            u\big|_{\Gamma} & = f \\
            u(x,y,0) & = u_0(x,y).
    \end{split}
\end{equation}

where $\Gamma=\partial\Omega$ is the domain's boundary.

### Numerical Method

To compute the derivatives we use Chebyshev differentiation matrix for spatial domain and for time domain we use Euler's Method [2].

In [1]:
# Python version 3.6.2
%matplotlib inline
import numpy as np # version 1.13.1
import ipywidgets as widgets # version 7.0.1
from scipy.interpolate import interp2d # version 0.19.1
from scipy.integrate import odeint
import matplotlib.pyplot as plt # version 2.0.2
from mpl_toolkits.mplot3d import Axes3D   
from matplotlib import cm

In [2]:
# Chebyshev differentiation matrix
def cheb(N):
    if N == 0:
        D = 0
        x = 1
        return D, x
    x = np.cos(np.pi * np.arange(N + 1) / N)
    c = np.hstack((2, np.ones(N - 1), 2)) * ((-1.)**np.arange(N + 1))
    X = np.tile(x, (N + 1, 1)).T
    dX = X - X.T
    D = np.outer(c, 1./c) / (dX + np.eye(N + 1))
    D = D - np.diag(np.sum(D.T, axis=0))
    return D,x

In [3]:
# Plot solution
def plot(t, u, v, a, i):
    # Finer mesh
    fine = np.linspace(-1, 1, 2*N)
    Xf, Yf = np.meshgrid(fine, fine)
    fu = interp2d(x, y, u[i].reshape(N+1, N+1), kind='cubic')
    fa = interp2d(x, y, a(x,y), kind='cubic')
    U = fu(fine, fine)  
    # Trick to remove oscillation of cubic interpolation
    tol = np.min(U) + 1e-2 # Tolerance 
    U[U <= tol] = tol # Remove values of oscillation
    A = fa(fine, fine)
    V1 = v[0](Xv, Yv)
    V2 = v[1](Xv, Yv)
    fig = plt.figure(figsize=(14, 8))      
    cont = plt.contourf(Xf, Yf, U, cmap=cm.jet, alpha=0.4)
    fig.colorbar(cont)
    if (not np.all(A == 0)):
        cont2 = plt.contour(Xf, Yf, A, cmap=cm.jet)
        fig.colorbar(cont2)
    if (not (np.all(V1 == 0) and np.all(V2 == 0))):
        plt.quiver(Xv, Yv, V1, V2, alpha=0.5)    
    plt.xlabel("x")
    plt.ylabel("y")
    plt.tight_layout()
    plt.title("$t$: " + "{:10.2f}".format(t[i]))
    plt.show()

In [4]:
# Helper to build reaction rate
# Gaussian basis
def G(x, y):
    return np.exp(-(x**2 + y**2))

# Superposition of gaussians based in https://commons.wikimedia.org/wiki/File:Scalar_field.png
def S(x, y):
    return G(2*x, 2*y) + 0.8 * G(2*x + 1.25, 2*y + 1.25) + 0.5 * G(2*x - 1.25, 4*y + 1.25) \
        - 0.5 * G(3*x - 1.25, 3*y - 1.25) + 0.35 * G(2*x + 1.25, 2*y - 1.25) \
        + 0.8 * G(x - 1.25, 3*y + 1.5) + 1.2 * G(x + 1.25, 3*y - 1.85)

In [6]:
# PDE's RHS
def F(W, t, mu, A, V1, V2):
    # Reshape W to Matrix
    W = W.reshape(N+1, N+1)
    diff = mu*(np.dot(W, D2x.T) + np.dot(D2y, W))    
    #conv = np.dot(np.dot(W, Dx.T), V1) + np.dot(W, np.dot(V1, Dx.T)) \
    #    + np.dot(np.dot(Dy, W), V2) + np.dot(W, np.dot(Dy, V2))
    #reac = np.dot(A, W)
    conv = np.dot(W, Dx.T) * V1 + np.dot(Dy, W) * V2 # np.dot(W, np.dot(V1, Dx.T)) \
         #+ np.dot(W, np.dot(Dy, V2))
    reac = A*W
    W = diff - conv + reac
    
    # Boundary conditions
    W[0,:] = np.zeros(N+1)
    W[-1,:] = np.zeros(N+1)
    W[:,0] = np.zeros(N+1)
    W[:,-1] = np.zeros(N+1)
    
    return W.flatten() # Flatten for odeint

In [7]:
# Solve PDE
def solvePDE(mu, dt, T, A, V1, V2, W):
    t = np.linspace(0, dt*T, T)
    U = odeint(F, W, t, args=(mu, A, V1, V2))
            
    return t, U

In [8]:
# Plot experiment using widgets
def plotExperiment(t, u, v, a):
    slider = widgets.IntSlider(
        value=0, 
        min=0, 
        max=len(t)-1, 
        step=1, 
        description='Time step:',
        continuous_update=False,
        readout=True,
        readout_format='d'
    )
    widgets.interact(plot, t=widgets.fixed(t), u=widgets.fixed(u), 
                     v=widgets.fixed(v), a=widgets.fixed(a), i=slider)

In [9]:
# Solve PDE for parameters
def experiment(param):
    A = param['a'](X, Y)
    V1, V2 = param['v'][0](X, Y), param['v'][1](X, Y)
    W = param['u0'](X, Y)  
    
    t, u = solvePDE(
        param['mu'], 
        param['dt'], 
        param['T'], 
        A, 
        V1, 
        V2,
        W.flatten() # Flatten initial condition for odeint
    )
    return t, u

In [10]:
# Fixed variables
N = 40

# x variable in [-1,1], Chebyshev
Nx = N 
Dx, x = cheb(Nx)
D2x = np.dot(Dx, Dx)

# y variable in [-1,1], Chebyshev
Ny = N
Dy, y = cheb(Ny) 
D2y = np.dot(Dy, Dy)

# Grids
X, Y = np.meshgrid(x,y)
Xv, Yv = np.mgrid[-1:1:21j, -1:1:21j]

In [11]:
# Reaction Rate
a = lambda x, y: 10*S(X, Y)

# Vectorial field
v1 = lambda x, y: 1
v2 = lambda x, y: np.sin(x**2 + y**2)

# Initial condition
u0 = lambda x, y: 1e3*np.exp(-40*((x+.5)**2 + (y+.5)**2))

## Experiments

### Only Diffusion

In [12]:
param1 = {
    'mu': .8,
    'dt': 1e-2,
    'T': 20,
    'a': np.vectorize(lambda x, y: 0),
    'v': (np.vectorize(lambda x, y: 0), np.vectorize(lambda x, y: 0)),
    'u0': u0
}

In [13]:
t1, u1 = experiment(param1)

In [14]:
plotExperiment(t1, u1, param1['v'], param1['a'])

interactive(children=(IntSlider(value=0, continuous_update=False, description='Time step:', max=19), Output())…

### Diffusion + Convection

In [15]:
param2 = {
    'mu': .8,
    'dt': 1e-2,
    'T': 20,
    'a': np.vectorize(lambda x, y: 0),
    'v': (v1, v2),
    'u0': u0
}

In [16]:
t2, u2 = experiment(param2)

In [17]:
plotExperiment(t2, u2, param2['v'], param2['a'])

interactive(children=(IntSlider(value=0, continuous_update=False, description='Time step:', max=19), Output())…

### Diffusion + Reaction

In [18]:
param3 = {
    'mu': .8,
    'dt': 1e-2,
    'T': 20,
    'a': a,
    'v': (np.vectorize(lambda x, y: 0), np.vectorize(lambda x, y: 0)),
    'u0': u0
}

In [19]:
t3, u3 = experiment(param3)

In [20]:
plotExperiment(t3, u3, param3['v'], param3['a'])

interactive(children=(IntSlider(value=0, continuous_update=False, description='Time step:', max=19), Output())…

### Diffusion + Convection + Reaction

In [27]:
param4 = {
    'mu': .01,
    'dt': 1e-2,
    'T': 200,
    'a': a,
    'v': (v1, v2),
    'u0': u0
}

In [28]:
t4, u4 = experiment(param4)

In [29]:
plotExperiment(t4, u4, param4['v'], param4['a'])

interactive(children=(IntSlider(value=0, continuous_update=False, description='Time step:', max=199), Output()…

## Conclusions

* The first experiment shows how the temperature diffuses over the domain without the effect of wind and fuel.
* In the second experiment we note the effect of vector field, where the heat moves in direction of flow.
* In the third experiment we note how the temperature moves to zones with high reaction rate.
* The last experiment shows the effect of reaction rate together with the vectorial field. The first elevates the temperature in zones with high reaction rate and the second one "moves" the heat, slightly increasing the temperature in the direction of the field. 

Qualitatively, this work shows interesting results since graphically the solutions seem to be consistent with reality, empirically demonstrating the possible effect of vector ("wind") and scalar ("fuel") field in the propagation of fire.

## References

1. Liu, W. (2009). Elementary feedback stabilization of the linear reaction-convection-diffusion equation and the wave equation (Vol. 66). Springer Science & Business Media.
2. Trefethen, L. N. (2000). Spectral methods in MATLAB (Vol. 10). Siam.